In [ ]:
from pyspark.sql.types import IntegerType

In [ ]:
# customersInputPath = "data/Customers.txt" # Useless for this program
purchaseInputPath = "data/Purchases.txt"
catalogInputPath = "data/ItemsCatalog.txt"

outputPath1 = "outPart1SQL/"
outputPath2 = "outPart2SQL/"

In [ ]:
# Define the dataframes associated with the input files

purchaseDF = spark.read.format("csv")\
            .option("header", False)\
            .option("inferSchema", True)\
            .load(purchaseInputPath)\
            .withColumnRenamed("_c0", "SaleTimestamp")\
            .withColumnRenamed("_c1", "Username")\
            .withColumnRenamed("_c2", "ItemID")\
            .withColumnRenamed("_c3", "SalePrice")


# Register the temporary table purchases 
purchaseDF.createOrReplaceTempView("purchases")




catalogDF = spark.read.format("csv")\
            .option("header", False)\
            .option("inferSchema", True)\
            .load(catalogInputPath)\
            .withColumnRenamed("_c0", "itemID")\
            .withColumnRenamed("_c1", "Name")\
            .withColumnRenamed("_c2", "Category")\
            .withColumnRenamed("_c3", "FirstTimeInCatalog")


# Register the temporary table catalog 
catalogDF.createOrReplaceTempView("catalog")

In [ ]:
#########################################
# PART 1
#########################################

In [ ]:
# Register a function that given a timestamp in the format "YYYY/MM/DD-HH:MM:SS" returns the year  
spark.udf.register("EXTRACTYEAR",\
                   lambda date: int(date.split("/")[0]),\
                   IntegerType())

In [ ]:
# Filter only the years 2020 and 2021 and cache this dataframe, so that it can be
# used for the second part
purchase20_21DF = spark.sql("SELECT * " \
                            +"FROM purchases " \
                            +"WHERE EXTRACTYEAR(SaleTimestamp)=2020 " \
                            +"OR  EXTRACTYEAR(SaleTimestamp)=2021")\
                        .cache()

# Register the temporary table purchase20_21 
purchase20_21DF.createOrReplaceTempView("purchase20_21")

In [ ]:
# Register a function that given a timestamp in the format "YYYY/MM/DD-HH:MM:SS" 
# and a year returns 1 or 0
# 1 if the year in the date is the one specified as a parameter
# 0 if the year in the date is not the one specified as a parameter
spark.udf.register("ISYEAR",\
                   lambda date, year: 1 if int(date.split("/")[0])==year else 0,\
                   IntegerType())

# You can define two UDFs instead of ISYEAR
# IS2020 that returns 1 if the year is 2020. Othewise, 0.
# IS2021 that returns 1 if the year is 2021. Othewise, 0.
#
# ISYEAR is more general and does not need to define two UDFs. Hovewer, the efficiency of 
# the solution based on ISYEAR or the other two UDFs is th same. 
# The solution based on ISYEAR is only more concise.

In [ ]:
# Use the user defined function to create two new "temporary attributes" for each record
# ISYEAR(SaleTimestamp, 2020) is 1 if the purchase is associated with the year is 2020. Otherwise, 0.
# ISYEAR(SaleTimestamp, 2021) is 1 if the purchase is associated with the year is 2021. Otherwise, 0.
#
# Then, use group by and sum in the having to count the number of purchases in 2020 and 2021
# for each item
selectedItemsPart1 = spark.sql("SELECT itemId " \
                               +"FROM purchase20_21 " \
                               +"GROUP BY itemId " \
                               +"HAVING SUM(ISYEAR(SaleTimestamp, 2020))>10000 " \
                               +"   AND SUM(ISYEAR(SaleTimestamp, 2021))>10000")

In [ ]:
# Store the result in the first output folder
selectedItemsPart1.write\
                .format("csv")\
                .option("header", False)\
                .save(outputPath1)

In [ ]:
#########################################
# PART 2
#########################################

In [ ]:
# Register a function that given a timestamp in the format "YYYY/MM/DD-HH:MM:SS" returns the month  
spark.udf.register("EXTRACTMONTH", \
                   lambda date: int(date.split("/")[1]), \
                   IntegerType())

In [ ]:
# Count the number of distinct customers for each item+month
# and filter only those groups item+month for which the distinct customers were >= 10

itemsMonthsMoreThan9DF = spark.sql("SELECT itemId, EXTRACTMONTH(SaleTimestamp) as month " \
                                   +"FROM purchase20_21 " \
                                   +"WHERE EXTRACTYEAR(SaleTimestamp)=2020 " \
                                   +"GROUP BY itemId, EXTRACTMONTH(SaleTimestamp) " \
                                   +"HAVING COUNT(DISTINCT Username)>=10")



# Register the temporary table itemsMonthsMoreThan9
itemsMonthsMoreThan9DF.createOrReplaceTempView("itemsMonthsMoreThan9");

In [ ]:
# Select the items, and their categories, removing those items for which there
# are more than 10 months with "many distinct customers"
selectedItemsAndCategoryPart2 = spark.sql("SELECT itemId, category " \
                                          +"FROM catalog " \
                                          +"WHERE FirstTimeInCatalog<'2020/01/01' " \
                                          +"  AND itemId NOT IN " \
                                          +" (SELECT itemId " \
                                          +"  FROM itemsMonthsMoreThan9 " \
                                          +"  GROUP BY itemId " \
                                          +"  HAVING COUNT(*)>10)")

In [ ]:
# Store the result in the second output folder
selectedItemsAndCategoryPart2.write\
                            .format("csv")\
                            .option("header", False)\
                            .save(outputPath2)